In [3]:
import numpy as np
import pandas as pd
import spotipy
import datetime
from spotipy.oauth2 import SpotifyOAuth

please note that this notebook is very rough, i am using it as a messy workspace for exploring spotipy and developing foundational functions that I will use more later. 

# WHAT AM I ACTUALLY DOING
i want to figure out some sort of system or interface that allows navigation through the user's playlists or listening history and gives a visualized breakdown of their content. from there i would like to make tools or preset playlist generation methods using some user-selected set of songs to generate a playlist or queue through some novel mean. this could incorporate NLP techniques to group or suggest songs by lyrical content or other language-based info.

currently i have infrastructure to gather data from a user's own playlists and/or history, my immediate next steps are:
* begin learning Flask (or something similar) and make an interface for navigating a user's listening history and selecting a set of "input" songs
* after Flask is understood and set up, figure out how to allow other user's to sign in and connect their spotify activity (currently this is all just running on my personal Spotify developer account, and only accesses my own personal data)
* explore spotipys built-in song suggestion functions and see how they could be useful
* last.fm API has tags, genres, more language or categorical song data. see if that can be accessed and utilized with some other model?

In [4]:
# Replace with your own credentials from the Spotify Developer Dashboard
client_id = "c5029ed3aca649949ef00903e6aabf91"
client_secret = "836208c86df64c4d98db67b6bc426ace"

# Initialize the SpotifyOAauth object with the required scopes and redirect URI
sp = spotipy.Spotify(
    auth_manager=SpotifyOAuth(
        client_id=client_id,
        client_secret=client_secret,
        redirect_uri="http://localhost:8000/callback",
        scope="user-read-recently-played,playlist-read-private,user-modify-playback-state,user-top-read",  # Add additional scopes as needed
    )
)


In [5]:
# parking lot for finished functions :)

def get_songlist_from_playlist(playlist:str) -> list:
    playlist = sp.playlist(playlist)
    songlist = []
    for i in range(len(playlist['tracks']['items'])):
        songlist.append(playlist['tracks']['items'][i]['track']['uri'])
    return songlist

def get_songlist_from_history(limit:int) -> list:
    history = sp.current_user_recently_played(limit=limit)
    songlist = []
    for i in range(len(history['items'])):
        songlist.append(history['items'][i]['track']['uri'])
    return songlist

def get_songlist_from_toptracks(limit:int, time:str) -> list:
    #time can be one of three strings, 'long_term' gives all recorded data, 'medium_term' gives 6 months, 'short_term' gives 4 weeks
    history = sp.current_user_top_tracks(limit=limit, offset=0, time_range=time)
    songlist = []
    for i in range(len(history['items'])):
        songlist.append(history['items'][i]['uri'])
    return songlist

def get_features_from_song(ID:str) -> dict:
    info = sp.track(ID)
    features = sp.audio_features(ID)[0]
    songdata = {'name':info['name'],
                   'uri':info['uri'],
                   'first_artist':info['artists'][0]['uri'], #will i ever need to process features or collab type songs? revisit this perhaps
                   'album':info['album']['uri'],
                   'duration_s':(info['duration_ms']/1000),
                   'release_date':info['album']['release_date'],  
                   'age':(datetime.date.today().year - int(info['album']['release_date'][:4])),
                   'popularity':info['popularity'],
                   'danceability':features['danceability'],
                   'energy':features['energy'],
                   'loudness':features['loudness'],
                   'speechiness':features['speechiness'],
                   'valence':features['valence'],
                   'instrumentalness':features['instrumentalness'],
                   'acousticness':features['acousticness'],
                   'liveness':features['liveness'],
                   'tempo':features['tempo'],
                   'mode':features['mode'],
                   'key':features['key'],
                   'time_signature':features['time_signature'],
                  }
    return songdata

def make_df_from_songlist(songlist:list):
    df = []
    for ID in songlist:
        df.append(get_features_from_song(ID))
    return pd.DataFrame(df)

In [6]:
playlists = sp.current_user_playlists()
names = [i['name'] for i in playlists['items']]
uris = [i['uri'] for i in playlists['items']]
user_playlists = [[names[i], uris[i]] for i in range(len(names))]
user_playlists
test = get_songlist_from_playlist(user_playlists[13][1])

In [7]:
test = make_df_from_songlist(test)
test

,name,uri,first_artist,album,duration_s,release_date,age,popularity,danceability,energy,loudness,speechiness,valence,instrumentalness,acousticness,liveness,tempo,mode,key,time_signature
0,Reelin' In The Years,spotify:track:1x1XQqhBViz4opcpwc7FVs,spotify:artist:6P7H3ai06vU1sGvdpBwDmE,spotify:album:6DlSUW5gmq6Byc3osKDJ2p,277.467,1972-11-01,51,75,0.494,0.540,-17.340,0.0348,0.723,0.000000,0.15400,0.0878,134.976,1,2,4
1,Southern Nights,spotify:track:7kv7zBjMtVf0eIJle2VZxn,spotify:artist:59hLmB5DrdihCYtNeFeW1U,spotify:album:0R7Uw1tUUbWlV53BGKfjQ4,180.026,1977-01-01,46,74,0.697,0.813,-8.890,0.0315,0.850,0.023200,0.09890,0.5830,95.301,0,11,4
2,Brown Eyed Girl,spotify:track:3yrSvpt2l1xhsV9Em88Pul,spotify:artist:44NX2ffIYHr6D4n7RaZF7A,spotify:album:7dsWupQRlFuhG8FGiQAUjC,183.306,1967-09,56,83,0.491,0.583,-10.964,0.0376,0.908,0.000000,0.18500,0.4060,150.566,1,7,4
3,Me and Julio Down by the Schoolyard,spotify:track:6vxHp3CDNo0afgKGp2yi1E,spotify:artist:2CvCyf1gEVhI0mX6aFXmVI,spotify:album:7npBPiCHjPj8PVIGPuHXep,164.813,1972-01-14,51,74,0.626,0.816,-10.593,0.1310,0.798,0.000002,0.23700,0.1040,105.307,1,9,4
4,Blister In The Sun,spotify:track:7jIAttgQTpLDoNtykIQXjH,spotify:artist:0rpMdBzQXf7aYRnu5fDBJy,spotify:album:0Ojio25IPzIrw1rUmgrhrG,145.706,1983-01-01,40,72,0.726,0.537,-8.896,0.1140,0.882,0.000000,0.31600,0.0707,96.889,1,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Come and Get Your Love - Single Version,spotify:track:7GVUmCP00eSsqc4tzj1sDD,spotify:artist:0w7HLMvZOHatWVbAKee1zF,spotify:album:5Gf5m9M6RiK2lkjpbP0xRu,205.933,1973-11-01,50,79,0.731,0.804,-7.621,0.0473,0.957,0.000280,0.17200,0.0685,107.383,0,11,4
96,Hey Jude - Remastered 2015,spotify:track:0aym2LBJBk9DAYuHHutrIl,spotify:artist:3WrFJ7ztbogyGnTHbHJFl2,spotify:album:7vEJAtP3KgKSpOHVgwm3Eh,425.653,2000-11-13,23,73,0.386,0.607,-7.700,0.0261,0.532,0.000014,0.01120,0.0880,147.207,1,10,4
97,More Than a Feeling,spotify:track:1QEEqeFIZktqIpPI4jSVSF,spotify:artist:29kkCKKGXheHuoO829FxWK,spotify:album:2QLp07RO6anZHmtcKTEvSC,285.133,1976,47,82,0.377,0.681,-8.039,0.0298,0.285,0.002300,0.00088,0.0504,108.789,1,7,4
98,Sunflower - Spider-Man: Into the Spider-Verse,spotify:track:3KkXRkHbMCARz0aVfEt68P,spotify:artist:246dkjvS1zLTtiykXe5h60,spotify:album:35s58BRTGAEWztPo9WqCIs,158.040,2018-12-14,5,87,0.760,0.479,-5.574,0.0466,0.913,0.000000,0.55600,0.0703,89.911,1,2,4


In [23]:
artists = list(test['first_artist'][0:2])
tracks = list(test['uri'][1:2])
genres = sp.artist(artists[0])['genres'][0]
artists, tracks, genres
#sp.recommendations(seed_artists = artists,seed_genres =  genres,seed_tracks = tracks, limit = 20)

(['spotify:artist:6P7H3ai06vU1sGvdpBwDmE',
  'spotify:artist:59hLmB5DrdihCYtNeFeW1U'],
 ['spotify:track:7kv7zBjMtVf0eIJle2VZxn'],
 'album rock')

In [25]:
seed_tracks = ['spotify:track:4uLU6hMCjMI75M1A2tKUQC', 'spotify:track:2Vv7yqqZQ3Z1ZA9C0Yp9my']
seed_genres = ['rock', 'pop']

# Get track recommendations
sp.recommendations(seed_tracks=seed_tracks, seed_genres=seed_genres, limit=10)

{'tracks': [{'album': {'album_type': 'ALBUM',
    'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/7Ey4PD4MYsKc5I2dolUwbH'},
      'href': 'https://api.spotify.com/v1/artists/7Ey4PD4MYsKc5I2dolUwbH',
      'id': '7Ey4PD4MYsKc5I2dolUwbH',
      'name': 'Aerosmith',
      'type': 'artist',
      'uri': 'spotify:artist:7Ey4PD4MYsKc5I2dolUwbH'}],
    'available_markets': ['AR',
     'AU',
     'AT',
     'BE',
     'BO',
     'BR',
     'BG',
     'CA',
     'CL',
     'CO',
     'CR',
     'CY',
     'CZ',
     'DK',
     'DO',
     'DE',
     'EC',
     'EE',
     'SV',
     'FI',
     'FR',
     'GR',
     'GT',
     'HN',
     'HK',
     'HU',
     'IS',
     'IE',
     'IT',
     'LV',
     'LT',
     'LU',
     'MY',
     'MT',
     'MX',
     'NL',
     'NZ',
     'NI',
     'NO',
     'PA',
     'PY',
     'PE',
     'PH',
     'PL',
     'PT',
     'SG',
     'SK',
     'ES',
     'SE',
     'CH',
     'TW',
     'TR',
     'UY',
     'US',
     'GB',
   

In [41]:
search = input("what song: ")
results = sp.search(search, limit=1)
print(get_features_from_song(results['tracks']['items'][0]['uri']))

what song:  tubthumping


{'name': 'Tubthumping', 'uri': 'spotify:track:22HYEJveCvykVDHDiEEmjZ', 'first_artist': 'Chumbawamba', 'album': 'Tubthumper', 'duration_s': 278.778, 'release_date': '1997-09-01', 'danceability': 0.348, 'energy': 0.88, 'loudness': -7.366, 'speechiness': 0.0978, 'instrumentalness': 0, 'acousticness': 0.128, 'liveness': 0.357, 'tempo': 101.185, 'mode': 1, 'key': 2, 'time_signature': 4}


In [14]:
test_playlist = user_playlists[12] #summer of jeff
test = sp.playlist(test_playlist[1])

In [30]:
#for parsing playlists, where test is a playlist's dict as directly retrieved by sp.playlist(ID)
test['uri'] #get uri
test['description']
test['images']
test['images'][0]['url'] #get direct url to cover image
test['name']
test['tracks']['total'] #number of tracks

test['tracks']['items'] #list of track info dicts

print(test['name'], ' has ', test['tracks']['total'], ' tracks')

#for parsing songs
song = test['tracks']['items'][0]
song['added_at']
song['track'].keys() #actual info dict
# includes ['album', 'artists', 'available_markets', 'disc_number', 'duration_ms', 'episode', 'explicit', 'external_ids', 'external_urls', 'href', 'id', 'is_local', 'name', 'popularity', 'preview_url', 'track', 'track_number', 'type', 'uri']
song['track']['album']['name'] 
song['track']['album']['uri']
song['track']['album']['release_date']
song['track']['name']
song['track']['uri']
song['track']['popularity'] #0-100, based on internal algorithm, measurement of number and recency of streams
song['track']['artists'][0]['name'] #name of the top listed artist for a song, can also call 'uri'
song['track']['duration_ms']/1000 #duration of song in seconds
song['track']

features = sp.audio_features(song['track']['uri'])[0] #get audio features, including 'danceability' 'energy' 'key' 'loudness' 'mode'
features['danceability'] #0-1, measure of songs beat strength, regularity, rhythm, and tempo 
features['energy'] #0-1, measure of the perceived intensity of a song, including its 'loudness', predictability, and tempo
features['loudness'] #average dB loudness of the song, typically falling from 0-60
features['key'] #key the song is in from 0-11, or -1 if none detected, higher value = higher average pitch of song
features['mode'] #modality, 0 = minor, 1 = major
features['speechiness'] #0-1, spoken-wordness of the track, 1 means no music, most songs are below 0.33
features['acousticness'] #0-1, 1 = more acoustic, 0 = more electronic
features['instrumentalness'] #0-1, 1 = more instrumental and less speech, 0 = vice versa
features['liveness'] #0-1, higher value means more likelihood the song is recorded live
features['tempo'] #average/estimated BPM of song
features['valence'] #0-1, musical 'positiveness' of track with 1 being more uplifting and 0 being more downbeat
features['time_signature'] #3-7, where x means the song has a x/4 signature

song['track']['album']['release_date']

TypeError: list indices must be integers or slices, not str

In [7]:


sp.recommendation(

,name,uri,first_artist,album,duration_s,release_date,danceability,energy,loudness,speechiness,instrumentalness,acousticness,liveness,tempo,mode,key,time_signature
0,Angelina,spotify:track:7Dt26KoNRmZpzlVwMe6LSP,spotify:artist:2gbT6GPXMis0OAkZbEQCYB,spotify:album:7zlf5gmgcFBUSLZpVsh9EE,101.053,2015-10-15,0.447,0.541,-6.527,0.0279,0.000001,0.386000,0.3380,101.070,1,1,4
1,Message,spotify:track:5XgNhKEQBR5JcpN4puaKJv,spotify:artist:6lcwlkAjBPSKnFBZjjZFJs,spotify:album:5CVwTpfAIK2vN9IxuG5C3f,159.960,2012-01-15,0.538,0.386,-8.054,0.0296,0.134000,0.270000,0.0819,140.685,0,6,4
2,The Chain - 2004 Remaster,spotify:track:5e9TFTbltYBg2xThimr0rU,spotify:artist:08GQAI4eElDnROBrJRGE0X,spotify:album:0BwWUstDMUbgq2NYONRqlu,270.213,1977-02-04,0.545,0.670,-8.810,0.0496,0.000822,0.009000,0.0451,151.553,1,9,4
3,High,spotify:track:3GaamyFHQYbQi1ykc4nV4o,spotify:artist:2JFTRDi5v7JtqoouVe1z5D,spotify:album:40cS75mUmnhW7W4L1ZCBLj,152.571,2019-05-15,0.545,0.434,-8.326,0.0292,0.000026,0.157000,0.0828,139.828,1,10,4
4,Small Poppies,spotify:track:0Rw5Go9iYcvxun5DhSdroI,spotify:artist:4OOlG5eBXSkSAAEeKjJb5Y,spotify:album:6PosCDToF5fOdEjTI5PsDX,419.906,2015-03-24,0.332,0.442,-9.055,0.0269,0.012800,0.004700,0.1150,170.717,1,7,3
5,Carry the Zero,spotify:track:3HMOMdRPywfouYx5B4PvaH,spotify:artist:3kbBWco9PZ5eSQsNScwG6U,spotify:album:2PKZhfPrgMmK5zug2pqdHA,343.893,1999-01-22,0.264,0.812,-5.651,0.0310,0.108000,0.000013,0.0962,94.095,1,4,4
6,All My Life,spotify:track:3xfoP9A6kYZ6sLp4LzjW1y,spotify:artist:12qS7lypMdpvASeNnFXtmN,spotify:album:1xOyUrDmQjLkYtWWv0iLf0,163.571,2017-11-10,0.553,0.704,-8.432,0.0293,0.003630,0.000231,0.1330,147.288,1,1,4
7,"Breathe in, Breathe Out",spotify:track:06fSbKiMP2Z7vzYG4WOBXB,spotify:artist:1S0vL284jxZYKtZQ2jsQ2X,spotify:album:0jolpH4eoj3kSEVg9mvZXr,170.333,2018-06-15,0.635,0.493,-8.571,0.0375,0.002430,0.063300,0.3520,122.037,1,2,4
8,Air Supply - 2021 Remaster,spotify:track:7KAXxQfUG4kzRivltbD1UY,spotify:artist:5NtMqQLCzdVvL7F8vFp3zM,spotify:album:0kmPn6M3cue7rec6Unw6BD,281.694,2009,0.676,0.705,-11.020,0.0327,0.001000,0.013900,0.1310,123.997,1,3,4
9,Shiggy,spotify:track:0Ed6Nry1j8FA1Ob4l7c2Wb,spotify:artist:4AGNJdJiVltImYk1UTLE0K,spotify:album:6GkbEFKMTv9kJ9XrnWBXiH,219.573,2018-08-24,0.554,0.674,-6.403,0.1090,0.000223,0.488000,0.1490,79.980,0,5,4


In [10]:
def get_spotify_track_details(track_name, artist_name):
    #client_credentials_manager = SpotifyClientCredentials(client_id='YOUR_CLIENT_ID', client_secret='YOUR_CLIENT_SECRET')
    #sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
    # Search for the input song
    search_results = sp.search(q=f'track:{track_name} artist:{artist_name}', type='track', limit=1)
    if not search_results['tracks']['items']:
        return None
    track = search_results['tracks']['items'][0]
    return track

def get_similar_tracks(track_id, limit=10):
    #client_credentials_manager = SpotifyClientCredentials(client_id='YOUR_CLIENT_ID', client_secret='YOUR_CLIENT_SECRET')
    #sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

    # Get audio features of the input track
    audio_features = sp.audio_features(track_id)
    if not audio_features:
        return None

    # Extract relevant audio features for similarity comparison
    input_features = audio_features[0]
    target_features = [input_features['danceability'],
                       input_features['energy'],
                       input_features['speechiness'],
                       input_features['acousticness'],
                       input_features['instrumentalness'],
                       input_features['liveness'],
                       input_features['valence'],
                       input_features['tempo']]

    # Get recommendations based on audio features
    recommended_tracks = sp.recommendations(seed_tracks=[track_id], limit=limit, target_audio_features=target_features)

    return recommended_tracks['tracks']

if __name__ == "__main__":
    input_song_name = 'i want wind to blow'
    input_artist_name = 'black countryoad'

    track_details = get_spotify_track_details(input_song_name, input_artist_name)

    if track_details:
        input_track_id = track_details['id']
        similar_tracks = get_similar_tracks(input_track_id, limit=10)

        print(f"Top 10 songs similar to '{input_song_name}' by {input_artist_name}:")
        for idx, track in enumerate(similar_tracks, start=1):
            print(f"{idx}. {track['name']} - {', '.join([artist['name'] for artist in track['artists']])}")
    else:
        print(f"No matching track found for '{input_song_name}' by {input_artist_name}.")


Top 10 songs similar to 'chaos space marine' by black country new road:
1. Spiritual Level Of Gang Shit - Soul Glo, McKinley Dixon, lojii
2. End of the Empire I-III - Arcade Fire
3. Bin Liner Fashion - Gilla Band
4. neon glow - glass beach
5. I Did Something Weird Last Night - Jeff Rosenstock
6. Everyone's Tongue - Horsey
7. Funeral Solution - Thee Oh Sees, Osees
8. When Life Hands You Problems - Cheekface
9. - - Porridge Radio
10. The Frontier Index - Silver Jews


In [16]:

lz_uri = 'spotify:artist:36QJpDe2go2KgaRleHCDTp'

results = sp.artist_top_tracks(lz_uri)

for track in results['tracks'][:10]:
    print('track    : ' + track['name'])
    print('audio    : ' + track['preview_url'])
    print('cover art: ' + track['album']['images'][0]['url'])
    print()

track    : Immigrant Song - Remaster
audio    : https://p.scdn.co/mp3-preview/f50f3416ea0807503e172ba87a92d8047d4f95da?cid=c5029ed3aca649949ef00903e6aabf91
cover art: https://i.scdn.co/image/ab67616d0000b27390a50cfe99a4c19ff3cbfbdb

track    : Stairway to Heaven - Remaster
audio    : https://p.scdn.co/mp3-preview/337c8d45ac66899bc2db9ef3ed9438fe1d035ab1?cid=c5029ed3aca649949ef00903e6aabf91
cover art: https://i.scdn.co/image/ab67616d0000b273c8a11e48c91a982d086afc69

track    : Whole Lotta Love - 1990 Remaster
audio    : https://p.scdn.co/mp3-preview/89f3dc45265602fa874c23d638538af0f2c6b56a?cid=c5029ed3aca649949ef00903e6aabf91
cover art: https://i.scdn.co/image/ab67616d0000b273fc4f17340773c6c3579fea0d

track    : Black Dog - Remaster
audio    : https://p.scdn.co/mp3-preview/78dd352fdbe8d2a867b72c5a71ac44ef6a462ab4?cid=c5029ed3aca649949ef00903e6aabf91
cover art: https://i.scdn.co/image/ab67616d0000b273c8a11e48c91a982d086afc69

track    : Rock and Roll - Remaster
audio    : https://p.scdn.